In [1]:
# Import PyDrive and associated libraries
# This only needs to be done once per notebook
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
# This only needs to be done once per notebook
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1M2S9M28Ruu0z5g_qEIuYYeOEidfC3Gox' # Check your own ID in GDrive
downloaded = drive.CreateFile({'id': file_id})

# Save file in Colab memory
downloaded.GetContentFile('tweet_data.csv') 

In [6]:
import pandas as pd
import re
import demoji
import string
import nltk
nltk.download('punkt')
from textblob import TextBlob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
df = pd.read_csv('tweet_data.csv')

In [8]:
df.sample(5)

,Unnamed: 0,Date,User,Tweet
10714,10714,2017-12-28 11:50:06+00:00,AdemCevik,@AdaletPlatformu @araskargo @ArasKargoDESTEK @...
8529,8529,2020-01-23 02:35:31+00:00,tkaraca,BOŞVEREN ŞİRKETLER https://t.co/3PMjxfKSzd #ar...
7905,7905,2020-04-29 10:53:27+00:00,frzndgsy,#araskargo #süratkargo 2 eleman fazla çalıştır...
7373,7373,2020-05-18 08:38:44+00:00,slmktrk,@ArasKargoDESTEK kargomun 10 gün oldu nerdeyse...
265,265,2022-08-06 06:48:55+00:00,ErkanGultekin58,@araskargo @ArasKargoDESTEK\nAldığınız emaneti...


In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stop_words = set(stopwords.words('turkish'))
print(stop_words)

{'şey', 'bu', 've', 'niye', 'de', 'az', 'yani', 'ki', 'hep', 'kim', 'niçin', 'acaba', 'biz', 'nerde', 'veya', 'kez', 'ama', 'ya', 'neden', 'bazı', 'diye', 'gibi', 'için', 'mı', 'sanki', 'mü', 'hem', 'eğer', 'en', 'ile', 'her', 'o', 'daha', 'siz', 'defa', 'biri', 'çok', 'hepsi', 'birşey', 'hiç', 'ise', 'mu', 'ne', 'şu', 'aslında', 'nerede', 'tüm', 'çünkü', 'nereye', 'da', 'nasıl', 'birkaç', 'belki'}


In [11]:
def cleanTweet(tweet):
    tweet = tweet.lower()
    tweet = demoji.replace(tweet,"")
    tweet = re.sub('\n',' ',tweet)
    tweet = re.sub(r'@\w+|\#','',tweet)
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    tweet = re.sub(r"http\s+|www\S+|https\S+","",tweet,flags=re.MULTILINE) # remove link 
    filtered_words = [word for word in TextBlob(tweet).words if word not in stop_words]
    return " ".join(filtered_words)

In [12]:
df['cleanTweet'] = df['Tweet'].map(cleanTweet)
df.sample(5)

,Unnamed: 0,Date,User,Tweet,cleanTweet
201,201,2022-08-18 21:23:07+00:00,Fehmi60461853,"Emanet edilen kargoyu çorbaya çeviren, ama hes...",emanet edilen kargoyu çorbaya çeviren hesabını...
10202,10202,2018-08-17 04:17:49+00:00,attila_y,@araskargo subeyi aradim sistem yok sonra aray...,subeyi aradim sistem yok sonra arayin dedi tel...
5735,5735,2020-09-29 14:12:57+00:00,ahmetcardak1234,@pismisseftali Bende sipariş ettim 3 gün içind...,bende sipariş ettim 3 gün içinde geldi yanlışn...
5222,5222,2020-11-19 05:50:50+00:00,pelinndemirciii,@Trendyol @ArasKargoDESTEK Aras kargo görüyoru...,aras kargo görüyorum herhangi bir desteğiniz o...
10868,10868,2017-11-06 11:28:51+00:00,hakki_ozturkk,#araskargo evde hasta var ve 7/24 evdeler adre...,araskargo evde hasta var 724 evdeler adreste k...


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1,2))
tm = tfidf.fit_transform(df["cleanTweet"])

In [16]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=10, random_state=0)
nmf.fit(tm)

NMF(n_components=10, random_state=0)

In [18]:
for index,topic in enumerate(nmf.components_):
    print(f"The top 20 words for topic # {index}")
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-20:]])
    print("\n")

The top 20 words for topic # 0
['yaşamadan anlamazsın', 'anlamazsın', 'yaşamadan', 'rezalet', 'vermeyen', 'hesabını vermeyen', 'durun araskargo', 'araskargo rezalet', 'pişmanlık', 'demektir', 'pişmanlık zarar', 'yalan pişmanlık', 'rezalet yalan', 'zarar demektir', 'emaneti', 'vermeyen dan', 'kıran', 'taşıdığı emaneti', 'emaneti kıran', 'kıran hesabını']


The top 20 words for topic # 1
['gelmeden paylaşın', 'engelledi haberi', 'çevirdi', 'çorbaya çevirdi', 'ni', 'haberi', 'kargoyu çorbaya', 'taşıdığı kargoyu', 'yanmasin', 'başkasi yanmasin', 'başkasi', 'çevirdi sonra', 'ni olsun', 'haberi ni', 'hesap', 'kırar', 'olsun dan', 'vermez', 'kırar hesap', 'hesap vermez']


The top 20 words for topic # 2
['almasaydın diyor', 'çi kırık', 'diyor dan', 'ver', 'çıkıyor teslim', 'kırık çıkıyor', 'araskargo ver', 'ver taşırken', 'kırıyor', 'taşırken kırıyor', 'diyor geldik', 'bulunamadın diyor', 'gelmeden adreste', 'etmedin', 'etmedin diyor', 'geldik kabul', 'kabul etmedin', 'bulunamadın', 'adreste 